# 인공신경망
- 뇌를 구성하는 신경세포, 즉 뉴런(neuron)의 동작 원리에 기초
- 뉴런의 기본 동작은 가지돌기에서 신호를 받아들이고, 신호가 축삭돌기를 지나 축삭말단으로 전달
- 하지만 신호가 너무 약해서 축삭말단까지 전달되지 않거나 강하게 전달됨
- 또 이런 신호가 다음 뉴런에 전달되고 이런 수십억개 뉴런들의 조합으로 인간은 살고있음
- 인공뉴런이란 입력값 X에 가중치(Weight)를 곱하고 편향(bias)을 더해 활성화함수를 거쳐 결과값y를 만들어내는 것
- W와 b의 값을 변경해가면서 적절한 값을 찾아내는 과정을 학습(learning) 또는 훈련(training)이라고 함
- 활성화 함수는 대표적으로 Sigmoid, ReLU, tanh가 있음

## 간단한 분류 모델 구현
- 패턴을 파악해 여러 종류로 구분하는 작업을 분류(classification)라고 함
- 아래 예제에서는 수치해석용 파이썬 라이브러리인 NumPy와 텐서플로를 사용함

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# 학습에 사용할 데이터 정의
x_data = np.array([
     [0,0]
    ,[1,0]
    ,[1,1]
    ,[0,0]
    ,[0,0]
    ,[0,1]
])

# 원-핫 인코딩(one-hot encoding)
- 데이터가 가질 수 있는 값들을 일렬로 나열한 배열로 만들고, 표현하려는 값의 인덱스만 1로 표기하고 나머지는 0으로 채우는 기법
- A, B, C의 분류가 있다고 했을시
    - A = [1,0,0]
    - B = [0,1,0]
    - C = [0,0,1]

In [3]:
y_data = np.array([
     [1,0,0] # A
    ,[0,1,0] # B
    ,[0,0,1] # C
    ,[1,0,0] # A
    ,[1,0,0] # A
    ,[0,0,1] # C
])

In [4]:
# 신경망 모델 구성
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([2, 3],-1, 1))
b = tf.Variable(tf.zeros([3]))

L = tf.add(tf.matmul(X, W),b)
L = tf.nn.relu(L)

# softmax 함수는 배열 내 결괏값들의 전체을 1로 만들어줌
model = tf.nn.softmax(L)

# 손실함수 교차 엔트로피(Cross-Entropy) 작성
# 교차 엔트로피란 예측값과 실제값 사이의 확률 분포 차이를 계산한 값
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

# 경사하강법으로 최적화
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
train_op = optimizer.minimize(cost)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [5]:
# 텐서플로 세션 초기화
init = tf.global_variables_initializer()


# 학습 진행
with tf.Session() as sess:
    sess.run(init)
    for step in range(100):
        sess.run(train_op, feed_dict={X: x_data, Y: y_data})
        if (step + 1) % 10 == 0: print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    # 학습결과 확인
    # argmax는 요소 중 가장 큰값의 인덱스를 찾아줌
    prediction = tf.argmax(model, axis = 1)
    target = tf.argmax(Y, axis=1)
    print('예측값: ', sess.run(prediction,feed_dict={X: x_data}))
    print('실제값: ', sess.run(target,feed_dict={Y: y_data}))
    
    # 정확도 출력
    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도 %.2f' % sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))

10 1.2734615
20 1.2183881
30 1.1797624
40 1.1547443
50 1.1328558
60 1.1173621
70 1.1083374
80 1.1017635
90 1.0986123
100 1.0986123
예측값:  [0 0 0 0 0 0]
실제값:  [0 1 2 0 0 2]
정확도 50.00


## 심층 신경망 구현
- 앞서 만든 모델에 가중치와 편향 추가

In [6]:
W1 = tf.Variable(tf.random_uniform([2, 10],-1, 1))
b1 = tf.Variable(tf.zeros([10]))

W2 = tf.Variable(tf.random_uniform([10, 3],-1, 1))
b2 = tf.Variable(tf.zeros([3]))

L1 = tf.add(tf.matmul(X, W1),b1)
L1 = tf.nn.relu(L1)

model = tf.add(tf.matmul(L1, W2),b2)

# 손실함수 교차 엔트로피(Cross-Entropy) 작성
# 교차 엔트로피란 예측값과 실제값 사이의 확률 분포 차이를 계산한 값
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

# 경사하강법으로 최적화
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
train_op = optimizer.minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
# 텐서플로 세션 초기화
init = tf.global_variables_initializer()


# 학습 진행
with tf.Session() as sess:
    sess.run(init)
    for step in range(100):
        sess.run(train_op, feed_dict={X: x_data, Y: y_data})
        if (step + 1) % 10 == 0: print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    # 학습결과 확인
    # argmax는 요소 중 가장 큰값의 인덱스를 찾아줌
    prediction = tf.argmax(model, axis = 1)
    target = tf.argmax(Y, axis=1)
    print('예측값: ', sess.run(prediction,feed_dict={X: x_data}))
    print('실제값: ', sess.run(target,feed_dict={Y: y_data}))
    
    # 정확도 출력
    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도 %.2f' % sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))

10 1.1244994
20 0.9011033
30 0.7798414
40 0.6913305
50 0.62037057
60 0.5644677
70 0.5176734
80 0.47700858
90 0.4406085
100 0.40823135
예측값:  [0 1 2 0 0 2]
실제값:  [0 1 2 0 0 2]
정확도 100.00
